<a href="https://colab.research.google.com/github/ankush2805/EIP/blob/master/Tests5/Copy_of_PersonAttrubutes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator


In [0]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

In [0]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

In [0]:
import keras
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa

ia.seed(1)

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        seq = iaa.Sequential([
                              iaa.Fliplr(0.1), # horizontal flips
                              iaa.Crop(percent=(0, 0.1)), # random crops
                              iaa.Sometimes(0.5,
                                            iaa.GaussianBlur(sigma=(0, 0.5))
                                            ),
                              # Strengthen or weaken the contrast in each image.
                              iaa.ContrastNormalization((0.75, 1.5)),
                              iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
                              iaa.Multiply((0.8, 1.2), per_channel=0.2),
                              iaa.Affine(
                                  scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                                  translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                                  rotate=(-25, 25),
                                  shear=(-8, 8))], random_order=True) # apply augmenters in random order

        images_aug = seq(images=image)
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return images_aug, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [0]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15)
train_df.shape, val_df.shape

In [0]:
train_df.head()

In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32)
valid_gen = PersonDataGenerator(train_df, batch_size=64, shuffle=False)

In [0]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

In [0]:
from keras.applications import InceptionResNetV2
backbone = InceptionResNetV2(
    weights=None, 
    include_top=False, 
    input_tensor=Input(shape=(224, 224, 3))
)

neck = backbone.output
neck = Flatten(name="flatten")(neck)
neck = Dense(512, activation="relu")(neck)


def build_tower(in_layer):
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.3)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

In [0]:
from keras.optimizers import Adam
# losses = {
# 	"gender_output": "binary_crossentropy",
# 	"image_quality_output": "categorical_crossentropy",
# 	"age_output": "categorical_crossentropy",
# 	"weight_output": "categorical_crossentropy",

# }
# loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0}
opt = Adam(lr=0.001)
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
    # loss_weights=loss_weights, 
    metrics=["accuracy"]
)

In [0]:
def plot_model_history(model_history):
    acc = ['gender_output_acc', 'image_quality_output_acc' , 'age_output_acc', 'weight_output_acc','bag_output_acc', 'footwear_output_acc','pose_output_acc',  'emotion_output_acc']
    val_acc = ['val_gender_output_acc', 'val_image_quality_output_acc' , 'val_age_output_acc', 'val_weight_output_acc','val_bag_output_acc', 'val_footwear_output_acc','val_pose_output_acc',  'val_emotion_output_acc']
    loss = ['gender_output_loss', 'image_quality_output_loss' , 'age_output_loss', 'weight_output_loss','bag_output_loss', 'footwear_output_loss','pose_output_loss',  'emotion_output_loss']
    val_loss = ['val_gender_output_loss', 'val_image_quality_output_loss' , 'val_age_output_loss', 'val_weight_output_loss','val_bag_output_loss', 'val_footwear_output_loss','val_pose_output_loss',  'val_emotion_output_loss']
    fig, axs = plt.subplots(8,2,figsize=(25,25))
    for i in range(8) :
      axs[i][0].plot(range(1,len(model_history.history[acc[i]])+1),model_history.history[acc[i]])
      axs[i][0].plot(range(1,len(model_history.history[val_acc[i]])+1),model_history.history[val_acc[i]])
      axs[i][0].set_title(acc[i])
      
      axs[i][0].set_xticks(np.arange(1,len(model_history.history[val_acc[i]])+1),len(model_history.history[val_acc[i]])/10)
      axs[i][0].legend(['train', 'val'], loc='best')
      # summarize history for loss
      axs[i][1].plot(range(1,len(model_history.history[loss[i]])+1),model_history.history[loss[i]])
      axs[i][1].plot(range(1,len(model_history.history[val_loss[i]])+1),model_history.history[val_loss[i]])
      axs[i][1].set_title(loss[i])
      
      axs[i][1].set_xticks(np.arange(1,len(model_history.history[loss[i]])+1),len(model_history.history[loss[i]])/10)
      axs[i][1].legend(['train', 'val'], loc='best')

    plt.show()

In [0]:
model_info  = model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=100,
    verbose=1
)
import matplotlib.pyplot as plt
plot_model_history(model_info)

In [0]:
model